In [235]:
from binascii import hexlify
from binascii import unhexlify
from hashlib import sha256
from hashlib import sha512
import hmac
from ecc import PrivateKey, S256Point, Signature
from helper import encode_base58_checksum, decode_base58, decode_base58_extended, hash160
from io import BytesIO, StringIO

N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

In [236]:
def extended_privkey(depth,finger_print,_index,chain_code,privkey, testnet=False):
    if testnet:
        version= 0x04358394.to_bytes(4,"big")
    else:
        version= 0x0488ade4.to_bytes(4,"big")
    depth=depth.to_bytes(1,"big")
    fingerprint =finger_print
    index = _index.to_bytes(4,"big")
    return version+depth+fingerprint+index+chain_code+b'\x00'+privkey

def deserialize_xprvk(s):
    version = s.read(4)
    depth = s.read(1)
    fingerprint = s.read(4)
    index= s.read(4)
    chain_code = s.read(32)
    s.read(1)
    privkey = s.read(32)
    return [version,depth, fingerprint,index,chain_code,privkey]


In [237]:
xtprvk = 'xprv9s21ZrQH143K31xYSDQpPDxsXRTUcvj2iNHm5NUtrGiGG5e2DtALGdso3pGz6ssrdK4PFmM8NSpSBHNqPqm55Qn3LqFtT2emdEXVYsCzC2U'
decoded =deserialize_xprvk( BytesIO(decode_base58_extended(xtprvk)))


In [238]:
for x in decoded:
    print(hex(int.from_bytes(x,"big")))

0x488ade4
0x0
0x0
0x0
0x60499f801b896d83179a4374aeb7822aaeaceaa0db1f85ee3e904c4defbd9689
0x4b03d6fc340455b363f51020ad3ecca4f0850280cf436c70c727923f6db46c3e


In [239]:
master_priv_key = decoded[-1]
chain_code = decoded[-2]

In [240]:
pub_key = PrivateKey(int.from_bytes(master_priv_key,"big")).point.sec()
finger_print = hash160(pub_key)[:4]
print(hex(int.from_bytes(finger_print,"big")))

0xbd16bee5


In [241]:
i=0

In [242]:
if i >= 2**31: hardened=True
else: hardened=False
    
if hardened:
    I= hmac.new(
                key = chain_code,
                msg=b'\x00' + master_priv_key + i.to_bytes(4,"big") ,
                digestmod=sha512).digest()

    
else:
    
    I= hmac.new(
                key = chain_code,
                msg=pub_key + i.to_bytes(4,"big") ,
                digestmod=sha512).digest()
    
print(f"I: {I}")

I: b'`\xe3s\x9c\xc2\xc3\x95\x0b|M\x7f2\xccP>\x13\xb9\x96\xd0\xf7\xa4V#\xd0\xa9\x14\xe1\xef\xa7\xf8\x11\xe0\xf0\x90\x9a\xff\xaa~\xe7\xab\xe5\xddN\x10\x05\x98\xd4\xdcS\xcdp\x9dZ\\,\xac@\xe7A/#/|\x9c'


In [243]:
I_L, I_R = I[:32], I[32:]

In [244]:
child_privkey_int = (int.from_bytes(I_L,"big") + int.from_bytes(master_priv_key,"big"))%N
hex(child_privkey_int)

'0xabe74a98f6c7eabee0428f53798f0ab8aa1bd37873999041703c742f15ac7e1e'

In [245]:
child_privkey = PrivateKey(child_privkey_int).wif(compressed= True)

In [246]:
child_chain_code = I_R

In [247]:
child_privkey

'L2ysLrR6KMSAtx7uPqmYpoTeiRzydXBattRXjXz5GDFPrdfPzKbj'

In [248]:
hex(int.from_bytes(child_chain_code,"big"))

'0xf0909affaa7ee7abe5dd4e100598d4dc53cd709d5a5c2cac40e7412f232f7c9c'

In [249]:
hex(int.from_bytes(PrivateKey(child_privkey_int).point.sec(),"big"))

'0x2fc9e5af0ac8d9b3cecfe2a888e2117ba3d089d8585886c9c826b6b22a98d12ea'

In [250]:
xtended_private_key = extended_privkey(1,finger_print,0,child_chain_code,child_privkey_int.to_bytes(32,"big"))
xtended_private_key = encode_base58_checksum(xtended_private_key)
xtended_private_key

'xprv9vHkqa6EV4sPZHYqZznhT2NPtPCjKuDKGY38FBWLvgaDx45zo9WQRUT3dKYnjwih2yJD9mkrocEZXo1ex8G81dwSM1fwqWpWkeS3v86pgKt'

In [251]:
 b'\xbd\x16\xbe\xe5',

(b'\xbd\x16\xbe\xe5',)

In [389]:
class Xtended_privkey:
    
    def __init__(self, depth, fingerprint, index, chain_code, private_key, testnet = False):
        """
        All data must be bytes
        """
        self.testnet = testnet
        self.version = None
        self.depth = depth
        self.fingerprint = fingerprint
        self.index = index
        self.chain_code = chain_code
        self.private_key = private_key
        self.private_key_wif = PrivateKey(child_privkey_int).wif()
        self.xtended_key = None
        
    def __repr__(cls):
        return cls.get_xtended_key()
    
    
    def get_child_xtended_key(self, i):
       
        pub_key = PrivateKey(int.from_bytes(self.private_key,"big")).point.sec()
       
        child_fingerprint = hash160(pub_key)[:4]
     
        if i >= 2**31: 
            hardened=True
            msg=b'\x00' + self.private_key + i.to_bytes(4,"big")
        else: 
            hardened=False
            msg=pub_key + i.to_bytes(4,"big") 

        I= hmac.new(
                    key = self.chain_code,
                    msg=msg,
                    digestmod=sha512).digest()
        
        I_L, I_R = I[:32], I[32:]
        
        child_privkey_int = (int.from_bytes(I_L,"big") + int.from_bytes(self.private_key,"big"))%N
        child_chain_code = I_R
        
        child_depth = (int.from_bytes(self.depth,"big")+1).to_bytes(1,"big")
        
        return Xtended_privkey(child_depth, child_fingerprint, i.to_bytes(4,"big"), 
                   child_chain_code, child_privkey_int.to_bytes(32,"big"),testnet = self.testnet)
        
        
        
    def get_child_from_path(self,path_string):
        child = self
        start = False
        begin = 0
        hardened = False
        for index,char in enumerate(path_string):
            
            if char == 'm' and index==0:
                continue
            elif char == '/':
                if start:
                    if hardened:
                        index = int(path_string[begin:index-1])+2**31
                        hardened = False
                    else:
                        index = int(path_string[begin:index])
                    child = child.get_child_xtended_key(index)
                    start = False
                continue
            elif char == "H" or char == "h":
                hardened = True
                continue
                
            elif not start:
                begin = index
                start = True
                
        index = int(path_string[begin:])
        child = child.get_child_xtended_key(int(char))
        return child
    
    
    def get_xtended_key(self):
        if self.testnet:
            version= 0x04358394.to_bytes(4,"big")
        else:
            version= 0x0488ade4.to_bytes(4,"big")
        
        pre_ser = version+self.depth+self.fingerprint+self.index+self.chain_code+b'\x00'+self.private_key
        return encode_base58_checksum(pre_ser)
    
    
    @classmethod
    def from_bip39_seed(cls, seed_int, testnet = False):
        I= hmac.new(
                    key = b"Bitcoin seed",
                    msg=seed_int.to_bytes(64,"big"),
                    digestmod=sha512).digest()
        return cls(depth=b'\x00', fingerprint=b'\x00'*4, index=b'\x00'*4, 
                   chain_code = I[32:], private_key=I[:32], testnet=testnet)
        
    
    @classmethod
    def parse(cls,xpk_string):
        testnet = None
        s = BytesIO(decode_base58_extended(xpk_string))
        version = s.read(4)
        if version == "04358394": 
            testnet = True
        elif version == "0488ade4": 
                testnet = False
        depth = s.read(1)
        fingerprint = s.read(4)
        index= s.read(4)
        chain_code = s.read(32)
        s.read(1)
        privkey = s.read(32)
        return cls(depth=depth, fingerprint=fingerprint, index=index, 
                   chain_code = chain_code, private_key=privkey, testnet=testnet)


In [390]:
string = "xprv9s21ZrQH143K3QTDL4LXw2F7HEK3wJUD2nW2nRk4stbPy6cq3jPPqjiChkVvvNKmPGJxWUtg6LnF5kejMRNNU3TGtRBeJgk33yuGBxrMPHi"
master = Xtended_privkey.parse(string)

In [391]:
master

xprv9s21ZrQH143K3QTDL4LXw2F7HEK3wJUD2nW2nRk4stbPy6cq3jPPqjiChkVvvNKmPGJxWUtg6LnF5kejMRNNU3TGtRBeJgk33yuGBxrMPHi

In [392]:
child = master.get_child_xtended_key(2**31)

In [393]:
child

xprv9uHRZZhk6KAJC1avXpDAp4MDc3sQKNxDiPvvkX8Br5ngLNv1TxvUxt4cV1rGL5hj6KCesnDYUhd7oWgT11eZG7XnxHrnYeSvkzY7d2bhkJ7

In [395]:
test = master.get_child_from_path("m/0H/1")

In [396]:
test

xprv9wTYmMFdV23N2TdNG573QoEsfRrWKQgWeibmLntzniatZvR9BmLnvSxqu53Kw1UmYPxLgboyZQaXwTCg8MSY3H2EU4pWcQDnRnrVA1xe8fs